Define smart collatz path based off odd collatz function:
    x mod 8 = 5, (x-1)/4, "L"
    x mod 4 = 3, (3x+1)/2, "2"
    x mod 8 = 1, (3x+1)/4, "4"
    
Objective: Perform sectioning on any binary number using its smart collatz path. (Doesn't seem to be useful in proving the conjecture - you would first need to carry out the path.)

In [1]:
#binary shortcuts

def binary(n):
    "Removes the leading 0b from the bin function."
    return str(bin(n))[2:]

def binaryAdd(str1, str2):
    "Adds two binary strings and returns the sum string."
    if not str1:
        return str2
    if not str2:
        return str1
    return binary(int(str1, 2) + int(str2, 2))

def binaryMultiply(str1, str2):
    "Multiplies two binary strings and returns the product string."
    if str1 and str2:
        return binary(int(str1, 2) * int(str2, 2))
    return ""

In [3]:
#implement the smart collatz traversal

def collDirection(string):
    "Performs one pass of the smart collatz function and returns a list containing the modified binary and direction used."
    direction = ""
    if string == "1":
        return [string, ""]
    if len(string) > 2 and string[-3:] == "101":
        direction = "L"
        string = string[:-3] + "1"
    elif len(string) > 1 and string[-2:] == "11":
        direction = "2"
        string = binaryAdd(binaryMultiply(string[:-2], "11"), "10") + "1"
    elif len(string) > 2 and string[-3:] == "001":
        direction = "4"
        string = binaryMultiply(string[:-3], "11") + "1"
    return [string, direction]

def easyColl(string, verbose=False):
    "Returns the smart collatz orbit of the number as a string with appended directions."
    history = ""
    if verbose:
        print(int(string, 2))
    while string is not "1":
        data = collDirection(string)
        if verbose:
            print(int(data[0], 2), data[1])
        string = data[0]
        history += data[1]
    return history

In [8]:
#identify depth identities

def smallBranches(n, depth, base=1):
    "Returns a list containing the n smallest numbers at specific depth from 1. 'x' denotes a branch stemming at multiple of 3."
    if depth == 0:
        return []
    x = [base]
    for i in range(1, n):
        x.append(4 * x[i-1] + 1)
    for i in range(2, depth + 1):
        for j, branch in enumerate(x):
            if branch == "x":
                continue
            if branch % 3 == 0:
                if i > 2:
                    x[j] = 4 * branch + 1
                    branch = x[j]
                else:
                    x[j] = "x"
                    continue
            exp = 1
            while (branch * (2 ** exp) - 1) % 3 != 0:
                exp += 1
            x[j] = (branch * (2 ** exp) - 1)//3
    return x

In [11]:
#derive finite automata transitions

def highlightDifference(str1, str2):
    "Returns a string labelling the binary and path differences between consecutive depth identities."
    idx = 0
    while idx < len(str1) and idx < len(str2) and str1[idx] is str2[idx]:
        idx += 1
    firstStrip = easyColl(str1 + "1").strip('L')
    pad = 'L'
    if firstStrip[-1] == '2':
        pad = 'LL'
    print(firstStrip + " (" + str1[idx:] + ") -> " + easyColl(str2 + "1").strip('L') + pad + " (" + str2[idx:] + ")")

def printDifferences(depth):
    "Prints all unique differences at specified depth."
    cycle = smallBranches(3 ** (depth - 1) + 3, depth)
    jump = 2
    i = 1
    while i + jump < len(cycle):
        highlightDifference(binary(cycle[i])[:-1], binary(cycle[i + jump])[:-1])
        i += jump
        jump = 3 - jump
        
printDifferences(3)

4L2 (0) -> 24LL (101)
24 () -> 22L (1)
22 () -> 4L4LL (11011000)
4L4 (0) -> 42L ()
42 (0) -> 44LL (100000)
44 (0) -> 4L2L (1000)


In [ ]:
#TODO: match warp nodes at state transitions to "nest" machines

def matchWarps(depth):
    cycle1 = smallBranches(3 ** (depth - 1) + 3, depth)
    cycle2 = smallBranches(3 ** (depth - 2) + 3, depth - 1)
    
    def getTransitions(cycle):
        
        jump = 2
        i = 1
        while i + jump < len(cycle):
            highlightDifference(binary(cycle[i])[:-1], binary(cycle[i + jump])[:-1])
            i += jump
            jump = 3 - jump